# Importing libraries

In [1]:
import time
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, box
from collections import defaultdict

import os
os.chdir(r"E:\Data challenge")

# Importing data

In [2]:
print("Loading clay data...")
data_clay = gpd.read_file(r"data\sol_argileux\ExpoArgile_Fxx_L93.shp")
data_clay = data_clay[['geometry', 'ALEA', 'NIVEAU']]

print(f"Clay data loaded: {len(data_clay)} features")
print(f"CRS: {data_clay.crs}")


Loading clay data...
Clay data loaded: 122222 features
CRS: EPSG:2154


In [3]:
data_drias = pd.read_csv(r"data\Drias_data\RCP_4.5.csv", sep=";", header=31)

# Create points

In [4]:
# Create points
print("\nCreating point geometries...")
points_gdf = gpd.GeoDataFrame(
    data_drias,
    geometry=[Point(lon, lat) for lon, lat in zip(data_drias['Longitude'], data_drias['Latitude'])],
    crs='EPSG:4326'  # Points are in WGS84
)

print(f"Points created: {len(points_gdf)}")



Creating point geometries...
Points created: 26943


# Filter points that doesn't belong to metropolitan France

In [5]:
# Transform points to match clay data CRS
print(f"\nTransforming points from EPSG:4326 to {data_clay.crs}...")
points_gdf = points_gdf.to_crs(data_clay.crs)

# Filter clay data to region of interest
print("\nFiltering clay data to points region...")
point_bounds = points_gdf.total_bounds
buffer_margin = 1000  # meters (since L93 is in meters)

min_x = point_bounds[0] - buffer_margin
min_y = point_bounds[1] - buffer_margin
max_x = point_bounds[2] + buffer_margin
max_y = point_bounds[3] + buffer_margin

data_clay = data_clay.cx[min_x:max_x, min_y:max_y]

print(f"Filtered clay data: {len(data_clay)} features")


Transforming points from EPSG:4326 to EPSG:2154...

Filtering clay data to points region...
Filtered clay data: 122216 features


# Create spatial index and create radius for buffer

In [6]:
# Create spatial index
print("Creating spatial index...")
sindex = data_clay.sindex
print("Spatial index created!")

Creating spatial index...
Spatial index created!


# Join drias and flood dataframe spatially

In [7]:
# Buffer distance in meters (L93 uses meters)
buffer_distance = 1  # 10 meters

# Process points
results = []
start_time = time.time()
print(f"\nProcessing {len(points_gdf)} points with buffer_distance={buffer_distance}m...")

for idx_num, (idx, point_row) in enumerate(points_gdf.iterrows()):
    
    # Progress update every 100 points
    if idx_num % 100 == 0:
        elapsed = time.time() - start_time
        if idx_num > 0:
            rate = idx_num / elapsed
            remaining = len(points_gdf) - idx_num
            est_remaining = remaining / rate / 60
            
            print(f"Point {idx_num}/{len(points_gdf)} ({idx_num/len(points_gdf)*100:.1f}%) | "
                  f"Rate: {rate:.2f} pts/sec | Elapsed: {elapsed:.1f}s | "
                  f"Est. remaining: {est_remaining:.1f}min")
    
    point = point_row.geometry
    lon = data_drias.loc[idx, 'Longitude']
    lat = data_drias.loc[idx, 'Latitude']
    
    # Create bounding box
    x, y = point.x, point.y
    bbox = box(x - buffer_distance, y - buffer_distance, 
               x + buffer_distance, y + buffer_distance)
    
    # Query spatial index
    possible_matches_idx = list(sindex.query(bbox))
    
    # Debug first point
    if idx_num == 0:
        print(f"\nFirst point found {len(possible_matches_idx)} candidates")
    
    if not possible_matches_idx:
        results.append({
            'longitude': lon,
            'latitude': lat,
            'alea': None,
            'niveau': None
        })
        continue
    
    # Get candidates
    possible_matches = data_clay.iloc[possible_matches_idx]
    
    # Collect all matching ALEA and NIVEAU values
    alea_list = []
    niveau_list = []
    
    for _, clay_row in possible_matches.iterrows():
        if point.distance(clay_row.geometry) <= buffer_distance:
            alea_list.append(clay_row['ALEA'])
            niveau_list.append(clay_row['NIVEAU'])
    
    results.append({
        'longitude': lon,
        'latitude': lat,
        'alea': max(set(alea_list), key=alea_list.count, default=None),
        'niveau': max(set(niveau_list), key=niveau_list.count, default=None)
    })

result_df = pd.DataFrame(results)

# Summary
total_time = (time.time() - start_time) / 60

print("\n" + "="*60)
print("COMPLETE!")
print("="*60)
print(f"Total processing time: {total_time:.1f} minutes")
print(f"Total points processed: {len(result_df)}")

print("\nFirst few rows:")
print(result_df.head())

# Save results
print("\nSaving results...")
result_df.to_pickle(r'data\Flood\clay_risk_results.pkl')
print("Results saved to 'data\Flood\clay_risk_results.pkl'")


Processing 26943 points with buffer_distance=1m...

First point found 0 candidates
Point 100/26943 (0.4%) | Rate: 923.75 pts/sec | Elapsed: 0.1s | Est. remaining: 0.5min
Point 200/26943 (0.7%) | Rate: 1033.26 pts/sec | Elapsed: 0.2s | Est. remaining: 0.4min
Point 300/26943 (1.1%) | Rate: 1017.27 pts/sec | Elapsed: 0.3s | Est. remaining: 0.4min
Point 400/26943 (1.5%) | Rate: 805.86 pts/sec | Elapsed: 0.5s | Est. remaining: 0.5min
Point 500/26943 (1.9%) | Rate: 670.55 pts/sec | Elapsed: 0.7s | Est. remaining: 0.7min
Point 600/26943 (2.2%) | Rate: 600.61 pts/sec | Elapsed: 1.0s | Est. remaining: 0.7min
Point 700/26943 (2.6%) | Rate: 552.66 pts/sec | Elapsed: 1.3s | Est. remaining: 0.8min
Point 800/26943 (3.0%) | Rate: 548.27 pts/sec | Elapsed: 1.5s | Est. remaining: 0.8min
Point 900/26943 (3.3%) | Rate: 533.16 pts/sec | Elapsed: 1.7s | Est. remaining: 0.8min
Point 1000/26943 (3.7%) | Rate: 517.35 pts/sec | Elapsed: 1.9s | Est. remaining: 0.8min
Point 1100/26943 (4.1%) | Rate: 512.72 pts/

# Save and export to csv

In [9]:
result_df.to_csv(r'data\Flood\clay_risk_results.csv')
print("Results saved to 'data\Flood\clay_risk_results.csv'")

Results saved to 'data\Flood\clay_risk_results.csv'


L'aléa d'inondation est composé de 4 sous aléa, par la suite nous regroupons ces aléa par probabilité d'occurences.